https://kernc.github.io/backtesting.py/doc/examples/Strategies%20Library.html
https://kernc.github.io/backtesting.py/doc/examples/Quick%20Start%20User%20Guide.html

In [101]:
import pandas as pd
import os

# import backtesting and yfinance

from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA
import yfinance as yf

# define backtesting strategy...simple SMA crossover
# see https://kernc.github.io/backtesting.py/doc/examples/Quick%20Start%20User%20Guide.html

# more strategies avalable!!!
# https://kernc.github.io/backtesting.py/doc/examples/Strategies%20Library.html

class SmaCross(Strategy):
    n1 = 10
    n2 = 20

    def init(self):
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)

    def next(self):
        if crossover(self.sma1, self.sma2):
            self.buy()
        elif crossover(self.sma2, self.sma1):
            self.sell()
            
            
# generate dataset compatible with backtesting strategy

def get_data(tickerSymbol, startdate='2010-1-1', enddate='2020-7-7'):
    # tickerSymbol = "NVTA"
    tickerData = yf.Ticker(tickerSymbol)
    tdf = tickerData.history(start=startdate, end=enddate).drop(['Dividends', 'Stock Splits'], axis=1)
    tdf.index.name = None
    return tdf

# run the backtest and plot the results

def backtest_results(ticker, start, end):
    tdf = get_data(ticker,startdate=start, enddate=end)
    bt = Backtest(tdf, SmaCross, cash=10000, commission=.002)
    output = bt.run()
    print(output)
    bt.plot()
    return output

## Get some of the top 'momentum' opportunitues

#### `xdays = days since the 50 sma crossed the 250 sma`

> should probably change these windows to be smaller

#### rev_spread is `(high revenue - low revenue)/average revenue` from yfinance data


In [66]:
momentum_stats = "/Users/mingay/coding/aws_mingquant/stocks/momentum_stats.txt"
momdf = pd.read_csv(momentum_stats, sep="\t")
momdf.columns = ['stock', 'xdays', 'rev_spread']
subdf = momdf[(momdf['xdays'] > 0) & (momdf['xdays'] < 20)].sort_values('rev_spread', ascending=False)
subdf.head()

,stock,xdays,rev_spread
190,UBER,4,0.624138
108,FND,18,0.347752
167,RAMP,4,0.188826
115,AMCR,6,0.139171
27,SFIX,4,0.117945


In [102]:
print([i for i in subdf.stock])

['UBER', 'FND', 'RAMP', 'AMCR', 'SFIX', 'NVTA', 'ORCL', 'UPWK', 'KDP', 'PANW', 'PS']


In [106]:
ticker = "KDP"
startdate='2020-01-01'
enddate='2020-7-7'
results = backtest_results(ticker, startdate, enddate)

Start                     2020-01-02 00:00:00
End                       2020-07-06 00:00:00
Duration                    186 days 00:00:00
Exposure [%]                          53.2258
Equity Final [$]                      12659.3
Equity Peak [$]                       13286.5
Return [%]                            26.5925
Buy & Hold Return [%]                0.527983
Max. Drawdown [%]                    -14.9037
Avg. Drawdown [%]                     -4.1279
Max. Drawdown Duration       83 days 00:00:00
Avg. Drawdown Duration       15 days 00:00:00
# Trades                                    4
Win Rate [%]                               50
Best Trade [%]                        15.5488
Worst Trade [%]                       -4.6339
Avg. Trade [%]                        4.97909
Max. Trade Duration          33 days 00:00:00
Avg. Trade Duration          25 days 00:00:00
Expectancy [%]                        8.36536
SQN                                  0.956002
Sharpe Ratio                      